#**Casting the Net:  Machine Learning for Phishing Website Detection**

##Abstract 

The goal of the project was to train a series of models to classify URLs as legimitimate or fraudulent (URLs that lead to phishing scams) and provide insights into the classification process. Decision Trees, Random Forests, and Neural networks were used as models while feature importances provided understanding as to how the model classifies samples. The Decision Tree and Random Forest models performed well with accuracies between 95% and 100%; The Neural Network implementation did not perform as well with accuracies maxing out at around 76%. Feature importance analysis revealed that certain features and feature categories played far more important roles in classifiying the URLs. Specifically, the structure of the filename and its associated directory in the URL as well as the information pertaining to the website, namely the hostname TTL, number of redirects, the time it took for the domain to be activated, etc.

##Introduction

Here you have to explain the problem that you are solving. Explain why it is important, and what are the main challenges. Mention previous attempts (add papers as references) to solve it. Mainly focus on the techniques closely related to our approach. Briefly describe your approach and explain why it is promising for solving the addressed problem. Mention the dataset and the main results achieved.

Phishing is a common source of fraud on the internet that can have
devastating effects on the lives of its victims. The goal of this project is to
produce a model that can quickly identify whether or not a URL presented to a
user is malicious as well as provide an intuitive understanding as to why the
URL in question is malicious (or not) by extracting insights from the model by examining feature importance (only performed on decision tree and random forest models in this report).  The dataset used is composed of **88,647**
training examples with **111 features** in total (the features are categorised into
implicit groups that correspond to specific parts of a URLs anatomy such as
the domain or protocol) [1]. 

This dataset is used to train a Decision Tree, Random Forest, and Neural Network to classify the URLs. Feature importance is then extracted from the Decision Tree and Random Forest models to provide insight into the importance of the datasets features in regards to classfication.

 The majority of existing attempts at phishing detection rely upon a mix of URL features as well as the content of the webpage(s) the URL redirects too, and often take a "big data" approach in regards to the dataset used; Specifically, the datasets are continuously supplemented via web scraping and similar techniques. One of the more interesting approaches supplemented the typical practice of URL feature extraction with a scoring system based on the URL search engine page rank as phishing websites often rank very low (versus the website it is trying to impersonate) [2]. Another interesting appraoch was taken by Google researchers were they used a combination of URL feature extraction, webpage content, whitelists and blacklists, and hosting details to inform their models [3].








##**Experimental Setup**

https://data.mendeley.com/datasets/72ptz43s9v/1

Total number of instances: 88,647 \\
Number of legitimate website instances (labeled as 0): 58,000 \\
Number of phishing website instances (labeled as 1): 30,647 \\
Total number of features: 111 \\

train size = 62052 samples (70% of dataset) \\
test size = 26595 samples (30% of the dataset)

**why use a imbalanced dataset?**

Due to the fact that most websites are legitimate rather than fraudulent, the imbalance reflects the reality of the problem space. While the code below can be configured to use a balanced or imbalanced dataset, the model performs better when the imbalanced dataset is used. 

**Models and Hyperparameters**

Decision Tree implemented with Sklearn

Random Forest implemented with Sklearn 

Neural Network (implemented with PyTorch) with the following parameters: 

* **learning rate**: 0.01

* **weight decay**: 0.00005 

* **momentum**: 0.87

* **batch size**: 64

##**Methodology**

1. process the data by separating the label column of the CSV file from the rest of the feature columns. 

2. Split the training data into training, validation, and test sets 

3. create Tensor copies of the aformentioned sets for use with the pytorch Neural Network. 

4. Train the Decision Tree and Random Forests and calculate their respective accuracies

5. Extract feature importances based on Mean Decrease in Impurity (MDI) and feature permutation for both the Decision Tree and Random Forest

6. Train Neural Network and calculate accuracy



##**Experimental Results**

In [129]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import sklearn.ensemble
import sklearn.inspection
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

##Data Processing

In [130]:
dataset = pd.read_csv('dataset_full.csv')

#feature labels and number of features for later use
feature_names = list(dataset.columns.values)[:111]
num_features = len(feature_names)

#split the csv into training data and labels
X, y = dataset.iloc[:,0:111], dataset['phishing']

#split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#creation of validation sets for use with feature permutation from the training sets
X_train, X_vali, y_train, y_vali = train_test_split(X_train, y_train, test_size=0.2)

#Conversion to PyTorch Tensors for use with the Neural Network implementation 
X_train_torch = torch.Tensor(X_train.values)
y_train_torch = torch.Tensor(y_train.values).long()

X_test_torch = torch.Tensor(X_test.values)
y_test_torch = torch.Tensor(y_test.values).long()

#**Decision Tree Implementation**

In [131]:
#train the model
tree = sklearn.tree.DecisionTreeClassifier(random_state=0).fit(X_train, y_train);

#training predictions and accuracy
y_pred = tree.predict(X_train)

training_accuracy = sklearn.metrics.accuracy_score(y_train, y_pred)

#test predictions and accuracy
y_pred = tree.predict(X_test)

test_accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)

print('{:.2%} training accuracy'.format(training_accuracy))
print('{:.2%} training error'.format(1-training_accuracy))
print('{:.2%} test accuracy'.format(test_accuracy))
print('{:.2%} test error'.format(1-test_accuracy))

100.00% training accuracy
0.00% training error
95.21% test accuracy
4.79% test error


##Feature Importances

###Mean Decrease In Impurity (MDI)

In [134]:
#extract the feature importances (MDI values) from the Decision Tree model object
importances = tree.feature_importances_
tree_importances = pd.Series(importances, index=feature_names)

In [ ]:
#plot the values with associated feature names
fig, ax = plt.subplots()
fig.set_size_inches(20,40)
tree_importances.plot.barh(ax=ax, align='center')
plt.title("Feature importances using 'Mean Decrease in Impurity (MDI)")
plt.ylabel("Mean decrease in impurity");

###Feature Permutation



In [ ]:
#feature importance mean calculation based on 10 runs (n_repeats)
result = permutation_importance(tree, X_vali, y_vali, n_repeats=10, random_state=42, n_jobs=-1)
#mean of the feature importances over the 10 runs
importances = result.importances_mean

In [ ]:
#plotting of the feature importances with associated labels
plt.figure(figsize=(20,40))
plt.barh(range(X.shape[1]), importances, align='center')
plt.yticks(range(X.shape[1]), feature_names)
plt.title('Feature Importance using Feature Permutation')
plt.xlabel('Feature Importance ')
plt.ylabel('Feature')
plt.show()

#**Random Forest Implementation**


In [136]:
#train the model
forest = sklearn.ensemble.RandomForestClassifier(random_state=0).fit(X_train, y_train);

#training predictions and accuracy
y_pred = forest.predict(X_train)

training_accuracy = sklearn.metrics.accuracy_score(y_train, y_pred)

#test predictions and accuracy
y_pred = forest.predict(X_test)

test_accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)

print('{:.2%} training accuracy'.format(training_accuracy))
print('{:.2%} training error'.format(1-training_accuracy))
print('{:.2%} test accuracy'.format(test_accuracy))
print('{:.2%} test error'.format(1-test_accuracy))

100.00% training accuracy
0.00% training error
96.80% test accuracy
3.20% test error


##Feature Importances

###Mean Decrease in Impurity (MDI)


In [137]:
#extract the feature importances (MDI values) from the Random Forest model object
importances = forest.feature_importances_
forest_importances = pd.Series(importances, index=feature_names)

In [ ]:
#plot the values with associated feature names
fig, ax = plt.subplots()
fig.set_size_inches(20,40)
forest_importances.plot.barh(ax=ax, align='center')
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")

###Feature Permutation

In [125]:
#feature importance mean calculation based on 10 runs (n_repeats)
result = permutation_importance(forest, X_vali, y_vali, n_repeats=10, random_state=42, n_jobs=-1)
#mean of the feature importances over the 10 runs
importances = result.importances_mean

In [ ]:
#plot the values with associated feature names
plt.figure(figsize=(20,40))
plt.barh(range(X.shape[1]), importances, align='center')
plt.yticks(range(X.shape[1]), feature_names)
plt.title('Feature Importance using Feature Permutation')
plt.xlabel('Feature Importance ')
plt.ylabel('Feature')
plt.show()

##**Neural Network Implementation**


In [ ]:
#create batches and shuffle
train_dataset = TensorDataset(X_train_torch, y_train_torch)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#network architecture
model = nn.Sequential(
    nn.Linear(111, 512),
    nn.ReLU(),
    nn.Linear(512, 384),
    nn.ReLU(),
    nn.Linear(384, 256),
    nn.ReLU(),
    nn.Linear(256, 192),
    nn.ReLU(),
    nn.Linear(192, 128),
    nn.ReLU(),
    nn.Linear(128, 96),
    nn.ReLU(),
    nn.Linear(96, 64),
    nn.ReLU(),
    nn.Linear(64, 48),
    nn.ReLU(),
    nn.Linear(48, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 2),
    nn.Sigmoid()
)

#optimizer and loss function with momentum and regularization (weight_decay)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.00005, momentum=0.87)
loss_fn = nn.CrossEntropyLoss()

num_epochs = 100

#training loop with mini-batching
for epoch in range(num_epochs):
    running_loss = 0.0
    running_corrects = 0
    
    #minibatching
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        labels = labels.view(-1)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, dim=1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels)
    
    #epoch loss and accuracy
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch+1, num_epochs, epoch_loss, epoch_acc*100))




#**References**

**[1]** Vrbančič, Grega (2020), “Phishing Websites Dataset”, Mendeley Data, V1, doi: 10.17632/72ptz43s9v.1

**[2]** "A novel Phishing classification based on URL features." IEEE Xplore. https://ieeexplore.ieee.org/abstract/document/5752505 (accessed Apr. 24, 2023).

**[3]** "Large-Scale Automatic Classification of Phishing Pages â Google Research." Google Research. https://research.google/pubs/pub35580/ (accessed Apr. 24, 2023).

